# Notebook #1


### Name: Tulsi Patel


### Date: 11.11.2021


### PID: 730392259


### Goal: Find average firing rate for Brain Observatory, Sst, Male, VISp, Static Gratings (SG) and Drifting Gratings (DG). 

*There are no Female SST VISl sessions. The Female SST VISp sessions have 0 low-contamination units. There are no Female WT/WT VISp and VISl sessions. (Disregard the Jupyter notebooks labeled "Female WT". They are not actually WT/WT. They are hybrids. 

## Protocol

### Starting code. 

In [1]:
import os

import numpy as np
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter

from allensdk.brain_observatory.ecephys.ecephys_project_cache import EcephysProjectCache
from allensdk.brain_observatory.ecephys.ecephys_session import (
    EcephysSession
)
from allensdk.brain_observatory.ecephys.visualization import plot_mean_waveforms, plot_spike_counts, raster_plot
from allensdk.brain_observatory.visualization import plot_running_speed

# tell pandas to show all columns when we display a DataFrame
pd.set_option("display.max_columns", None)

In [2]:
data_directory = '/Users/tulsipatel/local1/ecephys_cache_dir' 
# must be updated to a valid directory in your filesystem

manifest_path = os.path.join(data_directory, "manifest.json")

In [3]:
cache = EcephysProjectCache.from_warehouse(manifest=manifest_path)

In [4]:
units = cache.get_units()

In [5]:
sessions = cache.get_session_table()


print('Total number of sessions: ' + str(len(sessions)))

sessions.head()

Total number of sessions: 58


,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
721123822,2019-10-03T00:00:00Z,707296982,brain_observatory_1.1,125.0,M,Pvalb-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,444,2229,6,"[MB, SCig, PPT, NOT, DG, CA1, VISam, nan, LP, ..."
732592105,2019-10-03T00:00:00Z,717038288,brain_observatory_1.1,100.0,M,wt/wt,824,1847,5,"[grey, VISpm, nan, VISp, VISl, VISal, VISrl]"
737581020,2019-10-03T00:00:00Z,718643567,brain_observatory_1.1,108.0,M,wt/wt,568,2218,6,"[grey, VISmma, nan, VISpm, VISp, VISl, VISrl]"


In [6]:
filtered_sessions = sessions[(sessions.sex == 'M') & \
                             (sessions.full_genotype.str.find('Sst') > -1) & \
                             (sessions.session_type == 'brain_observatory_1.1') & \
                             (['VISl' in acronyms for acronyms in 
                               sessions.ecephys_structure_acronyms])]

filtered_sessions.head()

,published_at,specimen_id,session_type,age_in_days,sex,full_genotype,unit_count,channel_count,probe_count,ecephys_structure_acronyms
id,,,,,,,,,,
715093703,2019-10-03T00:00:00Z,699733581,brain_observatory_1.1,118.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,884,2219,6,"[CA1, VISrl, nan, PO, LP, LGd, CA3, DG, VISl, ..."
719161530,2019-10-03T00:00:00Z,703279284,brain_observatory_1.1,122.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,755,2214,6,"[TH, Eth, APN, POL, LP, DG, CA1, VISpm, nan, N..."
756029989,2019-10-03T00:00:00Z,734865738,brain_observatory_1.1,96.0,M,Sst-IRES-Cre/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,684,2214,6,"[TH, DG, CA3, CA1, VISl, nan, PO, Eth, LP, VIS..."


In [7]:
session_id = filtered_sessions.index.values[2]

session = cache.get_session_data(session_id)

In [8]:
units = session.units

print('Total number of units: ' + str(len(units)))

units.head()

Total number of units: 684


,L_ratio,presence_ratio,d_prime,waveform_halfwidth,cluster_id,firing_rate,peak_channel_id,silhouette_score,waveform_repolarization_slope,waveform_amplitude,snr,waveform_velocity_below,max_drift,nn_miss_rate,nn_hit_rate,waveform_velocity_above,waveform_spread,amplitude_cutoff,waveform_duration,waveform_PT_ratio,cumulative_drift,isolation_distance,isi_violations,waveform_recovery_slope,local_index_unit,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951814884,0.024771,0.99,3.555518,0.096147,6,9.492176,850126382,0.033776,0.673650,187.434780,3.342535,0.000000,45.64,0.016202,0.727333,0.000000,40.0,0.042404,0.151089,0.522713,463.86,46.750473,0.181638,-0.249885,5,NaN,2600.0,2.370440,1.765128,3.983333,1.866667,5.186364,0.244796,0.053998,0.033134,0.023705,61.923,0.612162,0.358,0.04,15.0,-0.154286,-0.005676,-0.228819,-0.212121,-0.310345,180.0,150.0,0.619716,9.763807e-01,0.507405,0.469507,0.447263,-5.000,4929,0.75,17.571944,11.057561,5.845194,8.805963,6.974832,NaN,0.0385,False,False,False,False,False,0.042644,False,0.011829,0.000012,0.062616,0.058606,0.039197,60,43,760640083,4,215.0,APN,8162.0,3487.0,6737.0,probeA,See electrode locations,29999.949611,1249.9979,True
951814876,0.001785,0.99,4.445414,0.206030,5,39.100557,850126382,0.108908,0.518633,129.686505,2.589717,0.000000,40.68,0.003756,1.000000,0.000000,50.0,0.097286,0.315913,0.652514,325.21,85.178750,0.004799,-0.143762,4,NaN,900.0,3.417573,0.704762,0.672690,0.803980,1.003055,0.137709,0.017675,0.015248,0.027334,72.222,0.678473,0.795,0.04,2.0,0.326587,0.231595,0.062157,0.068548,0.068853,315.0,150.0,0.000030,6.157503e-07,0.353566,0.674185,0.472870,45.556,5021,0.00,43.390301,44.709848,36.820288,44.885084,35.195889,NaN,0.1535,False,False,False,False,False,0.214051,False,0.001812,0.000003,0.002366,0.004308,0.002943,60,43,760640083,4,215.0,APN,8162.0,3487.0,6737.0,probeA,See electrode locations,29999.949611,1249.9979,True
951815032,0.035654,0.99,3.848256,0.096147,17,28.383277,850126398,0.096715,0.766347,207.380940,3.811566,-0.686767,40.01,0.014673,0.986000,-0.206030,80.0,0.015482,0.164824,0.484297,396.28,89.608836,0.007099,-0.255492,15,NaN,400.0,2.301810,1.408866,2.711028,1.714790,2.055258,0.173597,0.013665,0.007886,0.051909,85.000,0.768989,0.395,0.04,15.0,-0.026107,-0.220335,-0.345271,0.043011,-0.157445,135.0,120.0,0.827195,1.179300e-02,0.006750,0.867140,0.229617,5.000,4990,0.25,29.464485,28.829592,28.252666,28.025354,30.002900,NaN,0.0495,False,False,False,False,False,-0.005102,False,0.004121,0.006973,0.006743,0.018808,0.007783,140,43,760640083,12,215.0,APN,8129.0,3419.0,6762.0,probeA,See electrode locations,29999.949611,1249.9979,True
951815275,0.016783,0.99,3.065938,0.096147,30,5.709358,850126416,0.144249,0.628944,158.158650,2.918134,0.000000,33.32,0.003683,0.883598,0.686767,60.0,0.063807,0.178559,0.600600,374.82,48.114336,0.032317,-0.206676,27,NaN,200.0,3.264583,1.145098,2.160000,1.039456,2.604037,0.380119,0.018499,0.036959,0.072179,25.000,0.013071,0.786,0.04,15.0,-0.39781

In [9]:
V1_units = units[(units.ecephys_structure_acronym == 'VISl') &
                (units.isi_violations < 0.1)]

print('Total number of low-contamination V1 units: ' + str(len(V1_units)))


V1_units.head()

Total number of low-contamination V1 units: 20


,L_ratio,presence_ratio,d_prime,waveform_halfwidth,cluster_id,firing_rate,peak_channel_id,silhouette_score,waveform_repolarization_slope,waveform_amplitude,snr,waveform_velocity_below,max_drift,nn_miss_rate,nn_hit_rate,waveform_velocity_above,waveform_spread,amplitude_cutoff,waveform_duration,waveform_PT_ratio,cumulative_drift,isolation_distance,isi_violations,waveform_recovery_slope,local_index_unit,c50_dg,area_rf,fano_dg,fano_fl,fano_ns,fano_rf,fano_sg,f1_f0_dg,g_dsi_dg,g_osi_dg,g_osi_sg,azimuth_rf,mod_idx_dg,p_value_rf,pref_sf_sg,pref_tf_dg,run_mod_dg,run_mod_fl,run_mod_ns,run_mod_rf,run_mod_sg,pref_ori_dg,pref_ori_sg,run_pval_dg,run_pval_fl,run_pval_ns,run_pval_rf,run_pval_sg,elevation_rf,pref_image_ns,pref_phase_sg,firing_rate_dg,firing_rate_fl,firing_rate_ns,firing_rate_rf,firing_rate_sg,on_off_ratio_fl,time_to_peak_ns,pref_sf_multi_sg,pref_tf_multi_dg,pref_ori_multi_dg,pref_ori_multi_sg,pref_phase_multi_sg,image_selectivity_ns,pref_image_multi_ns,lifetime_sparseness_dg,lifetime_sparseness_fl,lifetime_sparseness_ns,lifetime_sparseness_rf,lifetime_sparseness_sg,probe_vertical_position,probe_horizontal_position,probe_id,channel_local_index,ecephys_structure_id,ecephys_structure_acronym,anterior_posterior_ccf_coordinate,dorsal_ventral_ccf_coordinate,left_right_ccf_coordinate,probe_description,location,probe_sampling_rate,probe_lfp_sampling_rate,probe_has_lfp_data
unit_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
951801653,0.044871,0.99,1.646617,0.425796,270,3.379305,850120400,0.098612,0.040582,27.102465,0.581829,0.932041,69.66,0.021877,0.478803,1.144612,140.0,0.063395,1.153769,0.010983,309.23,40.876865,0.052713,-0.007970,258,NaN,500.0,4.348718,1.500952,1.808182,0.711950,1.776175,0.866114,0.094683,0.235358,0.071968,28.000,0.085642,0.000,0.02,15.0,0.384615,-0.095588,-0.314590,-0.414634,0.078947,225.0,60.0,0.018153,0.703510,2.143763e-01,0.091290,0.766876,46.000,4923,0.25,2.439930,4.689472,3.219491,5.870642,3.832794,NaN,0.1145,False,False,False,False,False,0.239847,False,0.236090,0.002322,0.115578,0.118790,0.193413,2320,59,760640094,230,409.0,VISl,8921.0,2313.0,9333.0,probeD,See electrode locations,29999.905275,1249.996053,True
951801785,0.024859,0.99,3.498983,0.357119,283,12.380746,850120406,0.039057,0.200168,92.817465,2.161260,-0.343384,74.49,0.023785,0.902000,1.030151,60.0,0.002812,0.727973,0.300291,100.76,67.397396,0.003927,-0.029958,270,NaN,1600.0,2.458145,0.380819,0.623452,0.633333,1.040499,0.306357,0.022707,0.028442,0.048597,38.750,0.172222,0.274,0.02,15.0,-0.173729,0.105023,-0.005401,-0.155280,0.210728,45.0,60.0,0.570914,0.269670,9.624108e-01,0.528965,0.129518,23.125,5021,0.75,13.972412,8.153286,12.275444,8.093241,11.002129,NaN,0.1775,False,False,False,False,False,0.068661,False,0.015159,0.027304,0.010831,0.018012,0.026946,2340,11,760640094,233,409.0,VISl,8932.0,2277.0,9348.0,probeD,See electrode locations,29999.905275,1249.996053,True
951801773,0.016146,0.96,3.670702,0.206030,282,0.681763,850120406,NaN,0.378900,127.038210,2.669261,-0.343384,40.08,0.000302,0.691667,0.412060,60.0,0.020513,0.865327,0.143131,123.25,40.532477,0.080944,-0.036238,269,NaN,0.0,1.733333,0.891489,1.448889,0.800000,1.005566,1.688827,0.097735,0.330357,0.147620,NaN,1.009728,0.002,0.02,1.0,0.902778,0.215385,0.851282,-1.000000,0.666667,135.0,0.0,0.069015,0.042562,5.258895e-08,0.080516,0.021188,NaN,4979,0.50,0.101134,7.540457,1.108316,0.048246,0.193172,NaN,0.1155,False,False,False,False,False,0.634424,False,0.340459,0.766134,0.381744,0.870799,0.719192,2340,11,760640094,233,409.0,VISl,8932.0,2277.0,9348.0,probeD,See electrode locations,29999.905275,1249.996053,True
951801848,0.020434,0.99,2.771329,0.219765,289,2.061450,850120410,0.110593,0.204945,72.476235,1.847779,0.196219,68.05,0.003786,0.790936,1.167504,100.0,0.004015,0.618090,0.477174,295.83,42.885882,0.044266,-0.033611,276,NaN,400.0,4.147748,1.154568,2.294576,7.628571,1.155702,1.021083,0.161610,0.007519,0.074932,37.500,4.390417,0.006,0.32,4.0,-1.000000

## Find average firing rate for static gratings. 

In [10]:
print(V1_units.firing_rate_sg)

unit_id
951801653     3.832794
951801785    11.002129
951801773     0.193172
951801848     1.469437
951801836     2.124889
951801859     1.079097
951802013    11.425775
951802068     2.804321
951802055     1.330220
951802089    55.102568
951802098    17.111684
951802200    17.979625
951802233    25.470691
951802223     4.456272
951802334    23.814743
951802310    15.512356
951802288     6.182827
951802346     6.156849
951802409     4.325714
951802492     8.291064
Name: firing_rate_sg, dtype: float64


In [11]:
static_firing_rates=V1_units.firing_rate_sg 
static_average_firing_rate = sum(V1_units.firing_rate_sg)/len(V1_units.firing_rate_sg)
print(static_average_firing_rate)

10.983311434875636


## Find average firing rate for drifting gratings. 

In [12]:
print(V1_units.firing_rate_dg)

unit_id
951801653     2.439930
951801785    13.972412
951801773     0.101134
951801848     1.753170
951801836     2.911713
951801859     0.736003
951802013     7.524177
951802068     2.428281
951802055     1.191902
951802089    39.356050
951802098    15.518548
951802200    25.946496
951802233    20.901432
951802223    12.460693
951802334    17.228829
951802310    15.235796
951802288     6.425467
951802346     1.488421
951802409     6.828415
951802492    12.213947
Name: firing_rate_dg, dtype: float64


In [13]:
drifting_firing_rates=V1_units.firing_rate_dg 
drifting_average_firing_rate = sum(V1_units.firing_rate_dg)/len(V1_units.firing_rate_dg)
print(drifting_average_firing_rate)

10.333140788878778


## Remaining code: I was experimenting with another way to find the average firing rate. 

In [ ]:
presentation_table = session.stimulus_presentations[session.stimulus_presentations.stimulus_name == 'natural_scenes']


presentation_times = presentation_table.start_time.values
presentation_ids = presentation_table.index.values
presentation_table.head()

In [ ]:
units_of_interest = V1_units.index.values

In [ ]:
times = session.presentationwise_spike_times(
    stimulus_presentation_ids=presentation_ids,
    unit_ids=units_of_interest
)

times.head()

In [ ]:
average_firing_rate = '.0023476'

In [ ]:
length_of_time=times.time_since_stimulus_presentation_onset

In [ ]:
print(length_of_time.index.values[4])